<a href="https://colab.research.google.com/github/eldananyss21/assignment-3-classification/blob/main/code/assignment3_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3: Text Classification

In [2]:
# CLONE GITHUB REPO
!git clone https://github.com/eldananyss21/assignment-3-classification.git
%cd assignment-3-classification

Cloning into 'assignment-3-classification'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 47 (delta 6), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (47/47), 3.15 MiB | 3.36 MiB/s, done.
Resolving deltas: 100% (6/6), done.
/content/assignment-3-classification


In [12]:
!pip install datasets
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import Dataset
import torch
from sentence_transformers import SentenceTransformer
from sklearn import svm

print("Using GPU:", torch.cuda.is_available())

# IMPORT DATA
train = pd.read_csv("data/train.csv")
dev = pd.read_csv("data/dev.csv")
test = pd.read_csv("data/test.csv")

# ENCODE LABELS
label_encoder = LabelEncoder()
train['label'] = label_encoder.fit_transform(train['label'])
dev['label'] = label_encoder.transform(dev['label'])
label_names = label_encoder.classes_.tolist()
label2id = {name: i for i, name in enumerate(label_names)}
id2label = {i: name for i, name in enumerate(label_names)}

# TOKENIZE + CREATE DATASETS FOR TRACK 2 & TRACK 3
def tokenize_dataframes(model_name, train, dev, test, train_dev):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def tokenize_fn(examples):
        return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

    train_ds = Dataset.from_pandas(train).map(tokenize_fn, batched=True).rename_column("label", "labels")
    dev_ds = Dataset.from_pandas(dev).map(tokenize_fn, batched=True).rename_column("label", "labels")
    test_ds = Dataset.from_pandas(test).map(tokenize_fn, batched=True)
    train_dev_ds = Dataset.from_pandas(train_dev).map(tokenize_fn, batched=True).rename_column("label", "labels")

    return tokenizer, train_ds, dev_ds, test_ds, train_dev_ds

Using GPU: True


## Track 1: Machine Learning Technique

In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2')

# 1. TRAIN ON TRAIN SET → EVALUATE ON DEV SET
X_train = model.encode(train['text'].tolist(), show_progress_bar=True)
y_train = label_encoder.transform(train['label'])
X_dev = model.encode(dev['text'].tolist(), show_progress_bar=True)
y_dev = label_encoder.transform(dev['label'])

clf = svm.SVC()
clf.fit(X_train, y_train)
dev_preds = clf.predict(X_dev)
print("Track 1 Dev Macro F1:", f1_score(y_dev, dev_preds, average="macro"))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1000 [00:00<?, ?it/s]

Batches:   0%|          | 0/125 [00:00<?, ?it/s]

Track 1 Dev Macro F1: 0.42722599505243386


In [5]:
# 2. TRAIN ON TRAIN + DEV SETS → PREDICT ON TEST SET
train_dev = pd.concat([train, dev])
X = model.encode(train_dev['text'].tolist(), show_progress_bar=True)
y = label_encoder.transform(train_dev['label'])
X_test = model.encode(test['text'].tolist(), show_progress_bar=True)

clf.fit(X, y)
test_preds = clf.predict(X_test)
test_labels = label_encoder.inverse_transform(test_preds)

Batches:   0%|          | 0/1125 [00:00<?, ?it/s]

Batches:   0%|          | 0/125 [00:00<?, ?it/s]

In [6]:
pd.DataFrame({"id": test["id"], "label": test_labels}).to_csv("track_1_test.csv", index=False)

## Track 2: RoBERTa

In [13]:
model_name = "roberta-base"
tokenizer, train_ds, dev_ds, test_ds, train_dev_ds = tokenize_dataframes(model_name, train, dev, test, pd.concat([train, dev]))

model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           num_labels=len(label2id),
                                                           id2label=id2label,
                                                           label2id=label2id)

args = TrainingArguments(output_dir = "code/results_roberta",
                         eval_strategy="epoch",
                         save_strategy="epoch",
                         learning_rate=1e-5,
                         per_device_train_batch_size=16,
                         per_device_eval_batch_size=16,
                         num_train_epochs=3,
                         weight_decay=0.01,
                         load_best_model_at_end=True,
                         metric_for_best_model="macro_f1",
                         fp16=True,
                         logging_steps=50,
                         save_total_limit=1)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"macro_f1": f1_score(labels, preds, average="macro")}

trainer = Trainer(model=model,
                  args=args,
                  train_dataset=train_ds,
                  eval_dataset=dev_ds,
                  tokenizer=tokenizer,
                  compute_metrics=compute_metrics,
                  callbacks=[EarlyStoppingCallback(early_stopping_patience=2)])

Map:   0%|          | 0/32000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-13-894daf6c0676>:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model,


In [14]:
import os
os.environ["WANDB_DISABLED"] = "true"

trainer.train()
trainer.evaluate()

Epoch,Training Loss,Validation Loss,Macro F1
1,1.335100,1.327373,0.427500
2,1.234100,1.312950,0.441897
3,1.161100,1.325728,0.449491


{'eval_loss': 1.3257278203964233,
 'eval_macro_f1': 0.4494908936765253,
 'eval_runtime': 13.0163,
 'eval_samples_per_second': 307.306,
 'eval_steps_per_second': 19.207,
 'epoch': 3.0}

In [16]:
roberta_preds = trainer.predict(test_ds).predictions.argmax(-1)
labels = [id2label[i] for i in roberta_preds]
pd.DataFrame({"id": test["id"], "label": labels}).to_csv("track_2_test.csv", index=False)

## Track 3: XLNet

In [21]:
model_name = "xlnet-base-cased"
tokenizer, train_ds, dev_ds, test_ds, train_dev_ds = tokenize_dataframes(model_name, train, dev, test, pd.concat([train, dev]))

model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           num_labels=len(label2id),
                                                           id2label=id2label,
                                                           label2id=label2id)

args = TrainingArguments(output_dir="code/results_xlnet",
                         eval_strategy="epoch",
                         save_strategy="epoch",
                         learning_rate=1e-5,
                         per_device_train_batch_size=32,
                         per_device_eval_batch_size=16,
                         num_train_epochs=2,
                         weight_decay=0.01,
                         load_best_model_at_end=True,
                         metric_for_best_model="macro_f1",
                         fp16=False,
                         logging_steps=50,
                         save_total_limit=1)

trainer = Trainer(model=model,
                  args=args,
                  train_dataset=train_ds,
                  eval_dataset=dev_ds,
                  tokenizer=tokenizer,
                  compute_metrics=compute_metrics,
                  callbacks=[EarlyStoppingCallback(early_stopping_patience=2)])

Map:   0%|          | 0/32000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-21-5fc5abaa853c>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model,


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

trainer.train()
trainer.evaluate()

Epoch,Training Loss,Validation Loss,Macro F1
1,1.365300,1.366394,0.422305


In [ ]:
xlnet_preds = trainer.predict(test_ds).predictions.argmax(-1)
xlnet_labels = [id2label[i] for i in xlnet_preds]
pd.DataFrame({"id": test["id"], "label": xlnet_labels}).to_csv("track_3_test.csv", index=False)

NameError: name 'train_dev_ds' is not defined